# Graph

## What is Graph?

Graph is an **abstract data structure** in computer science

> “A set of objects, and the connections between them, are naturally expressed as a graph”

<img src="./img/graphs.jpg" alt="drawing" width="300" height="400"/>

### How to Represent Graph?

#### 1. Adjacency List

